In [1]:
# 1 Importing Libraries 
import os
import csv
import numpy as np
import pickle
import re
import json
from tqdm import tqdm

### Options

In [2]:
output_path = r"C:\Users\natal\Desktop\Korpus\debug_1812"
#output_path = r'C:\Users\natal\Desktop\Korpus\run_lsk_201219'
#output_path = r'D:\Natalie\13_korpus\run_bote_210216'
output_path = r'D:\Natalie\13_korpus\run_mag_210216'

# Load ML LSB

In [3]:
eval_pre=[]
debug_info=[]
classification=np.ndarray((0,))


for file in os.listdir(output_path):
    if file.startswith('temp_lsb'):
        with open(os.path.join(output_path, file), 'rb') as fileo:
            print(os.path.join(output_path, file))
            data = pickle.load(fileo)
            eval_pre += data["eval_pre"]
            debug_info += data["debug_info"]

            classification = np.concatenate((classification, data["eval_pred"]))


D:\Natalie\13_korpus\run_mag_210216\temp_lsb_0.pkl


In [4]:
saetze=[]
debug_info_pos=[]
for sen, clas, info in zip(eval_pre, classification, debug_info):
    if clas:
        print("positive result")
        print(sen)
        saetze.append(sen)
        debug_info_pos.append(info)
print(len(saetze))
print(len(debug_info_pos))

positive result
Schließlich haben wir bis jetzt zwar zähneknirschend aber anerkennend genickt wenn nach dieser Frage immer gleich die im Kanton Uri liegende Bergwiese Rütli zur Hand war
positive result
In diesem Jahrhundert als zum ersten Mal Vor und Nachnamen benutzt wurden und der Fluss Muota in der fruchtbaren Ebene des Talkessels Schwyz Fluch und Segen zugleich war
positive result
Zur Linken wandert der Blick über die malerischen Ausläufer von Schwyz ins Tal hinunter bis nach Brunnen und an den Vierwaldstättersee
positive result
 An der Umfriedungsmauer des Maihofs sehe ich acht Bienenkästen im warmen Licht der Frühlingssonne stehen leises Summen erfüllt die warme Luft
positive result
Nicht ganz so viele Jahre hat der Bödmeren Urwald im Bereich des Pragelpasses auf dem Buckel
positive result
Er war übrigens der Erste der ein Auto hatte im Muotathal und ich bin mit ihm durch die Wiesen und Wälder gefahren
positive result
 Im Bisistal hatten wir dann oben am Berg unser Hauptmotiv
pos

# Load Topo Pickles

In [5]:
sentences_winfo = []

for file in os.listdir(output_path):
    if file.startswith('temp_topo'):
        with open(os.path.join(output_path, file), 'rb') as fileo:
            print(os.path.join(output_path, file))
            data_frompickle = pickle.load(fileo)
            sentences_winfo += data_frompickle["sentences_winfo"]

D:\Natalie\13_korpus\run_mag_210216\temp_topo_0.pkl


In [6]:
def preprocess(sentence):
    return re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)  

In [7]:
sentences_winfo.sort(key=lambda x:preprocess(x["sentence"]))
debug_info_pos.sort(key=lambda x:x["sentence"])

In [8]:
print("sentences_winfo", len(sentences_winfo))
print("debug_info", len(debug_info_pos))
print("sentences", len(eval_pre))
print("sentences positive", len(saetze))
print(len(list(set(eval_pre))))
print(sum(classification))

sentences_winfo 3635
debug_info 248
sentences 911
sentences positive 248
907
248.0


In [9]:
#with open("tmp2.json", "w", encoding="utf-8") as file:
 #   json.dump(sentences_winfo, file, indent=4, ensure_ascii=False)

In [10]:
len(debug_info_pos)

248

In [11]:
results = []

debug = False
index = 0

not_found_list = []
for di in tqdm(debug_info_pos):
    sentence_to_find = di["sentence"]
    while True:
        #if index == len(debug_info_pos): 
        test_sentence = preprocess(sentences_winfo[index]["sentence"])
        
        if debug:
            print("--------", test_sentence)
            print("--------", sentence_to_find)
            print()
        if test_sentence == sentence_to_find:
            di["topo_infos"] = sentences_winfo[index]["topo_infos"]
            di["lsb"].sort(key= lambda x:x["ls-wort"])
            #print("--- found", test_sentence)
            last_word_dict = di["lsb"][0]
            new_lsb_list = [last_word_dict]
            for ls_word_dict in di["lsb"][1:]:
                if last_word_dict != ls_word_dict:
                    new_lsb_list.append(ls_word_dict)
                    last_word_dict = ls_word_dict
            di["lsb"] = new_lsb_list
            results.append(di)
            break
        
        index += 1
        
print(len(results))
print(len(debug_info_pos))
assert len(results) == len(debug_info_pos)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 248/248 [00:00<00:00, 7925.96it/s]

248
248


In [12]:
print(type(results[0]))
print(results[0])

<class 'dict'>
{'sentence': ' 1601 ZUM ERSTEN MAL HEISST ES RIGI KULM Der Stadtschreiber von Luzern Renward 48 Cysat erwähnt Kulm als den Namen des höchsten Gipfels der Rigi', 'lsb': [{'ls-wort': 'Gipfels', 'type': 'lsb'}], 'topo_infos': [{'topo': 'Kulm', 'info': [{'uuid': '{25912830-DE2A-44EA-BFBD-4B62474C4CC9}', 'objektart': 'Hauptgipfel', 'geom': 'PKT', 'coords': (2679518.836, 1212280.605, 1797.3652)}]}, {'topo': 'Rigi', 'info': [{'uuid': '{22435B69-8496-4610-AAFE-6D480E492A6A}', 'objektart': 'Haltestelle Bahn', 'geom': 'PKT', 'coords': (2678670.216, 1211912.883, 1603.4762)}, {'uuid': '{41B85DCA-4A79-4672-B5FB-557BDE4B4A23}', 'objektart': 'Massiv', 'geom': 'PLY', 'coords': (2681802.892, 1209422.853, 641.54855)}]}]}


In [13]:
##### Für jeden Satz eine ID hinzufügen!!

In [14]:
index=1

for sentence_info in results:
    sentence_info['ID']=index
    index+=1

with open(f"{output_path}/sents_w_info.json", 'w', encoding="utf-8") as output_file:
    json.dump(results, output_file, indent=4, ensure_ascii=False)